In [37]:
import pandas as pd
import glob

# Define path pattern to load all "education.csv" files in the 'data' folder
file_pattern = "data/*education.csv"
all_files = glob.glob(file_pattern)

# Initialize empty list to collect data frames
df_list = []

# Process each education file
for file in all_files:
    # Extract year from the file name (assuming the format 'YYYYeducation.csv')
    year = file.split('/')[-1][:4]

    # Load the CSV file
    df = pd.read_csv(file)

    # Drop "Population 25 years and over" and the nine rows following it
    pop_25_index = df[df.iloc[:, 0] == "Population 25 years and over"].index
    if not pop_25_index.empty:
        df = df.drop(index=range(pop_25_index[0], pop_25_index[0] + 10)).reset_index(drop=True)
    
    # Identify the first column name dynamically (the label column)
    label_column = df.columns[0]

    # Keep only the first column (label) and columns ending with "!!Female!!Estimate"
    columns_to_keep = [label_column] + [col for col in df.columns if col.endswith("!!Female!!Estimate")]
    df = df[columns_to_keep]
    
    # List to store processed data
    selected_data = []

    # Process each age group section to calculate weighted averages
    current_population = None
    total_population = 0  # To accumulate total population for the age group
    for index, row in df.iterrows():
        label = row[label_column]

        # Check if row represents a population group (e.g., "Population 18 to 24 years")
        if "Population" in label:
            # Extract population number from label and set as current population group
            try:
                total_population = int(label.split()[1]) if label.split()[1].isdigit() else None
            except ValueError:
                total_population = None
            current_population = total_population  # Set total for this age group
        elif current_population and "Percent" not in label:
            # Calculate weighted value for educational attainment percentages based on the total population
            for col in df.columns[1:]:  # Exclude the label column
                # Extract the state name from the column title
                state = col.split("!!")[0]

                # Get the percentage as a string and convert to a numeric type
                percent_str = row[col]
                try:
                    percent = float(str(percent_str).replace("%", "").strip())
                except ValueError:
                    percent = None  # Set as None if conversion fails

                scaled_value = (percent / 100) * total_population

                selected_data.append({
                    "Label": label,
                    "Year": year,
                    "State": state,
                    "Female Estimate": scaled_value,
                    "Population": total_population
                })

    # Convert selected_data list into a DataFrame and append to df_list
    temp_df = pd.DataFrame(selected_data)
    df_list.append(temp_df)

# Combine all data frames into a single data frame
combined_df = pd.concat(df_list, ignore_index=True)

                                       Label  Year          State  \
0                                Black alone  2022        Montana   
1                                Black alone  2022        Wyoming   
2             High school graduate or higher  2022        Montana   
3             High school graduate or higher  2022        Wyoming   
4                Bachelor's degree or higher  2022         Alaska   
...                                      ...   ...            ...   
1107             Bachelor's degree or higher  2017  West Virginia   
1108             Bachelor's degree or higher  2017        Wyoming   
1109             Bachelor's degree or higher  2017       Delaware   
1110             Bachelor's degree or higher  2017   South Dakota   
1111             Bachelor's degree or higher  2017        Wyoming   

      Female Estimate  Population  
0              432.90          65  
1              616.20          65  
2              395.20          65  
3              605.15      

,Label,Year,State,Female Estimate,Population
0,Black alone,2022,Montana,432.90,65
1,Black alone,2022,Wyoming,616.20,65
2,High school graduate or higher,2022,Montana,395.20,65
3,High school graduate or higher,2022,Wyoming,605.15,65
4,Bachelor's degree or higher,2022,Alaska,478.40,65


In [56]:
import glob
import pandas as pd

# Define path pattern to load all "race.csv" files with a year prefix in the 'data' folder
file_pattern = "data/*race.csv"
all_files = glob.glob(file_pattern)

# Initialize an empty list to collect data frames
df_list = []

# Loop through all matching files
for file_path in all_files:
    # Extract the year from the file name (assuming it is the first part of the file name)
    year = file_path.split('/')[-1].split('race')[0]
    
    try:
        # Load the CSV file, skipping the first two lines and using comma as the delimiter
        df = pd.read_csv(file_path, skiprows=2, delimiter=',')
        
        # Add the year column
        df['Year'] = year
        
        # Drop columns with specific names if they exist
        columns_to_drop = ['American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander', 'Total', 'Footnotes']
        df = df.loc[:, ~df.columns.str.contains('|'.join(columns_to_drop), na=False)]
        
        # Remove rows with "United States" in the first column
        df = df[~df.iloc[:, 0].str.strip().str.lower().eq("united states")]
        
        # Remove rows where "White" is NaN
        df = df.dropna(subset=["White"])
        
        # Append the cleaned dataframe to the list
        df_list.append(df)
    
    except pd.errors.ParserError:
        print(f"Could not parse {file_path}. Skipping this file.")
    except Exception as e:
        print(f"An error occurred with file {file_path}: {e}")

# Concatenate all dataframes in the list
final_df = pd.concat(df_list, ignore_index=True)

# Display the merged dataframe
print(final_df)


          Location  White  Black  Hispanic  Asian Multiple Races  Year
0          Alabama  0.660  0.265     0.041  0.012          0.017  2016
1           Alaska  0.614  0.029     0.067  0.060          0.068  2016
2          Arizona  0.555  0.040     0.309  0.032          0.023  2016
3         Arkansas  0.730  0.153     0.073  0.014          0.022  2016
4       California  0.375  0.054     0.390  0.142          0.033  2016
..             ...    ...    ...       ...    ...            ...   ...
411     Washington  0.675  0.038     0.130  0.089           0.05  2019
412  West Virginia  0.925  0.032     0.014  0.008          0.018  2019
413      Wisconsin  0.811  0.062     0.071  0.030           0.02  2019
414        Wyoming  0.838   0.01     0.101  0.006          0.021  2019
415    Puerto Rico  0.013  0.001     0.982    NaN          0.003  2019

[416 rows x 7 columns]


In [76]:
# Load the CSV file and examine its structure
file_path = 'data/SPM.csv'

# Reload the CSV file as a single column and then split based on the tab delimiter
df = pd.read_csv(file_path)

df.head()

,Year,States,Estimate
0,2022,Alabama,16.3
1,2022,Alaska,10.8
2,2022,Arizona,12.3
3,2022,Arkansas,16.6
4,2022,California,11.7


In [73]:
import pandas as pd

# Load the CSV file and examine its structure
file_path = 'data/foreignborn2022.csv'

# Reload the CSV file as a single column and then split based on the tab delimiter
df = pd.read_csv(file_path, header=None)

# Split the single column by tab to separate into "State" and "Percent"
df[['State', 'Percent']] = df[0].str.split('\t', expand=True)

# Drop the original combined column
df = df[['State', 'Percent']]

# Display the first few rows to confirm the split
df.head()


,State,Percent
0,California,26.70
1,New Jersey,23.50
2,New York,22.70
3,Florida,21.70
4,Nevada,18.90
